### 1. Importing libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

### 2. Importing data

In [2]:
# Set path to project folder

path = r'/Users/timovic/Desktop/Climate Analysis - battle station/02 Data'

In [3]:
# Import dataframe

df_all = pd.read_csv(os.path.join(path, 'Prepared Data', 'all_merged.csv'))
df_ihdi = pd.read_csv(os.path.join(path, 'Original Data', 'ihdi.csv'))           

In [4]:
df_all.shape

(6045, 10)

In [5]:
df_ihdi.shape

(195, 12)

### 3. Wrangling data

#### a. Converting from wide to long

In [6]:
df_ihdi.head()

,Country,IHDI_2010,IHDI_2011,IHDI_2012,IHDI_2013,IHDI_2014,IHDI_2015,IHDI_2016,IHDI_2017,IHDI_2018,IHDI_2019,IHDI_2020
0,Afghanistan,0.312,0.315,0.323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Angola,NaN,NaN,NaN,NaN,NaN,0.395,0.407,0.409,0.41,0.410,0.409
2,Albania,0.657,0.669,0.696,0.704,0.711,0.714,0.717,0.720,0.72,0.724,0.709
3,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Drop all null values

df_ihdi.dropna(inplace=True)

In [8]:
# Convert df using pandas melt

df_ihdi_melt = df_ihdi.melt(id_vars=["Country"], 
             value_vars=["IHDI_2010", "IHDI_2011", "IHDI_2012", "IHDI_2013", "IHDI_2014", 
                         "IHDI_2015", "IHDI_2016", "IHDI_2017", "IHDI_2018", "IHDI_2019", "IHDI_2020"],
             var_name="Year", value_name="IHDI")

In [12]:
# Convert df using pandas wide to long

df_ihdi_long = pd.wide_to_long(df_ihdi, 
                     stubnames=["IHDI"], 
                     i=["Country"], 
                     j="Year", 
                     sep="_", 
                     suffix="\d+").reset_index()

In [10]:
df_ihdi_melt.head()

,Country,Year,IHDI
0,Albania,IHDI_2010,0.657
1,Argentina,IHDI_2010,0.702
2,Armenia,IHDI_2010,0.673
3,Australia,IHDI_2010,0.846
4,Austria,IHDI_2010,0.843


In [13]:
df_ihdi_long.head()

,Country,Year,IHDI
0,Albania,2010,0.657
1,Argentina,2010,0.702
2,Armenia,2010,0.673
3,Australia,2010,0.846
4,Austria,2010,0.843


In [14]:
# Group long df by country and then year

df_ihdi_long = df_ihdi_long.sort_values(['Country', 'Year'], ascending=[True,True])

In [15]:
# Reset index

df_ihdi_long.reset_index(drop=True, inplace=True)

In [16]:
# Rename columns to prepare for merge

df_ihdi_long.rename(columns={'Country' : 'country', 'Year' : 'year', 'IHDI' : 'ihdi'}, inplace=True)

In [17]:
df_ihdi_long.head()

,country,year,ihdi
0,Albania,2010,0.657
1,Albania,2011,0.669
2,Albania,2012,0.696
3,Albania,2013,0.704
4,Albania,2014,0.711


### 4. Merging IHDI data with df_all 

In [18]:
df_all.head()

,Unnamed: 0,country,year,hdi,gni,co2_consumption,happiness,hdi_groups,undp_regions,continent
0,0,Afghanistan,1990,0.273,2684.550019,NaN,NaN,Low,SA,Asia
1,1,Afghanistan,1991,0.279,2276.289409,NaN,NaN,Low,SA,Asia
2,2,Afghanistan,1992,0.287,2059.868084,NaN,NaN,Low,SA,Asia
3,3,Afghanistan,1993,0.297,1525.533426,NaN,NaN,Low,SA,Asia
4,4,Afghanistan,1994,0.292,1087.961890,NaN,NaN,Low,SA,Asia


In [19]:
df_complete = pd.merge(df_all, df_ihdi_long, how='left', on=['country', 'year'])

In [22]:
df_complete.tail(10)

,Unnamed: 0,country,year,hdi,gni,co2_consumption,happiness,hdi_groups,undp_regions,continent,ihdi
6035,6035,Zimbabwe,2011,0.535,3154.375903,0.821565,NaN,Medium,SSA,Africa,0.381
6036,6036,Zimbabwe,2012,0.557,3618.629526,0.911292,NaN,Medium,SSA,Africa,0.400
6037,6037,Zimbabwe,2013,0.567,3632.111591,0.911311,NaN,Medium,SSA,Africa,0.410
6038,6038,Zimbabwe,2014,0.576,3644.856047,0.933605,NaN,Medium,SSA,Africa,0.420
6039,6039,Zimbabwe,2015,0.582,3638.532892,0.959957,4.6100,Medium,SSA,Africa,0.445
6040,6040,Zimbabwe,2016,0.588,3606.750671,0.825971,4.1930,Medium,SSA,Africa,0.450
6041,6041,Zimbabwe,2017,0.594,3728.918785,0.712244,3.8750,Medium,SSA,Africa,0.456
6042,6042,Zimbabwe,2018,0.602,3864.012419,0.834868,3.6920,Medium,SSA,Africa,0.463
6043,6043,Zimbabwe,2019,0.601,3674.564482,0.787526,3.6630,Medium,SSA,Africa,0.463
6044,6044,Zimbabwe,2020,0.600,3654.289051,0.741153,3.2992,Medium,SSA,Africa,0.463


### 5. Exporting data

In [23]:
df_complete.to_csv(os.path.join(path, 'Prepared Data', 'all_merged.csv'))